# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import sys
import time
import multiprocessing
import pprint 

from datetime import datetime, timedelta

from Packages.NoveltyDetection.setup.noveltyDetectionConfig import CONFIG
from SAENoveltyDetectionAnalysis import SAENoveltyDetectionAnalysis
from Functions.telegrambot import Bot

num_processes = multiprocessing.cpu_count()

my_bot = Bot("lisa_thebot")

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

training_params = {
    "Technique": "StackedAutoEncoder",
    "TechniqueParameters": {
        "allow_change_weights": False #False
    },
    "DevelopmentMode": True,
    "DevelopmentEvents": 1600,
    "NoveltyDetection": True,
    "InputDataConfig": {
        "database": "4classes",
        "n_pts_fft": 1024,
        "decimation_rate": 3,
        "spectrum_bins_left": 400,
        "n_windows": 1,
        "balance_data": True
    },
    "OptmizerAlgorithm": {
        "name": "Adam",
        "parameters": {
            "learning_rate": 0.001,
            "beta_1": 0.90,
            "beta_2": 0.999,
            "epsilon": 1e-08,
            "learning_decay": 1e-6,
            "momentum": 0.3,
            "nesterov": True
        }
    },
    "HyperParameters": {
        "n_folds": 4,#10, #4,
        #"n_epochs": 300,#500,
        "pretraining_n_epochs": 300,
        "finetuning_n_epochs": 300,
        "n_inits": 1,#1, #2,
        "batch_size": 128,#128, #256,
        "kernel_initializer": "uniform",
        "bias_initializer": "ones", #zeros
        "encoder_activation_function": "sigmoid",
        "decoder_activation_function": "sigmoid",
        "classifier_output_activation_function": "sigmoid",
        "norm": "mapstd",
        "metrics": ["accuracy"],
        "loss": "mean_squared_error",
        "dropout": False,
        "dropout_parameter": 0.00,
        "regularization": None, #None
        "regularization_parameter": 0.00 #0.00
    },
    "callbacks": {
        "EarlyStopping": {
            "patience": 30,
            "monitor": "val_loss"
        }
    }
}
analysis = SAENoveltyDetectionAnalysis(parameters=training_params,
                                       model_hash="",
                                       load_hash=False, load_data=True, verbose=True)
all_data, all_trgt, trgt_sparse = analysis.getData()

SAE = analysis.createSAEModels()

trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

Using TensorFlow backend.


[+] Time to read data file: 8.032801866531372 seconds
Qtd event of A is 12939
Qtd event of B is 29352
Qtd event of C is 11510
Qtd event of D is 23760

Biggest class is B with 29352 events
Total of events in the dataset is 77561
Balacing data...
Reading from C:\Users\vinic\Workspace\LPS\SonarAnalysis\Results\NoveltyDetection\4_folds_cross_validation_balanced_data_dev.jbl
Reading from C:\Users\vinic\Workspace\LPS\SonarAnalysis\Results\NoveltyDetection\4_folds_cross_validation_balanced_data_dev.jbl


In [2]:
pp = pprint.PrettyPrinter(indent=1)
print(analysis.model_hash)
print(analysis.getBaseResultsPath())
pp.pprint(analysis.parameters)

354a72b3e4ebfb12896c02ee30c130cde48d1f8f2703b6e61c596dded2d1c7c6
C:\Users\vinic\Workspace\LPS\SonarAnalysis\Results\NoveltyDetection\StackedAutoEncoder\outputs\354a72b3e4ebfb12896c02ee30c130cde48d1f8f2703b6e61c596dded2d1c7c6
{'DevelopmentEvents': 1600,
 'DevelopmentMode': True,
 'HyperParameters': {'batch_size': 128,
                     'bias_initializer': 'ones',
                     'classifier_output_activation_function': 'sigmoid',
                     'decoder_activation_function': 'sigmoid',
                     'dropout': False,
                     'dropout_parameter': 0.0,
                     'encoder_activation_function': 'sigmoid',
                     'finetuning_n_epochs': 300,
                     'kernel_initializer': 'uniform',
                     'loss': 'mean_squared_error',
                     'metrics': ['accuracy'],
                     'n_folds': 4,
                     'n_inits': 1,
                     'norm': 'mapstd',
                     'pretraining_n_ep

### Perform the training of the model

In [5]:
for inovelty in range(len(analysis.class_labels)):
    startTime = time.time()
    
    data=trn_data[inovelty]
    trgt=trn_trgt[inovelty]
    trainingType = "normal" #foldSweep, neuronSweep, normal, layerSweep
    analysis.train(layer=1,
                   inovelty=inovelty,
                   fineTuning=True,
                   trainingType=trainingType,
                   hidden_neurons=[400],
                   neurons_variation_step=50,
                   numThreads=2,
                   model_hash=analysis.model_hash)
    
    duration = str(timedelta(seconds=float(time.time() - startTime)))
    print("The training of the model for novelty class {0} took {1} to be performed\n".format(analysis.class_labels[inovelty], duration))

python C:\Users\vinic\Workspace\LPS\SonarAnalysis\Packages\NoveltyDetection\StackedAutoEncoders\sae_train.py --layer 1 --novelty 0 --finetunning 1 --threads 2 --type normal --hiddenNeurons 400 --neuronsVariationStep 50 --modelhash 354a72b3e4ebfb12896c02ee30c130cde48d1f8f2703b6e61c596dded2d1c7c6
The training of the model for novelty class A took 0:01:21.231113 to be performed

python C:\Users\vinic\Workspace\LPS\SonarAnalysis\Packages\NoveltyDetection\StackedAutoEncoders\sae_train.py --layer 1 --novelty 1 --finetunning 1 --threads 2 --type normal --hiddenNeurons 400 --neuronsVariationStep 50 --modelhash 354a72b3e4ebfb12896c02ee30c130cde48d1f8f2703b6e61c596dded2d1c7c6
The training of the model for novelty class B took 0:00:45.259048 to be performed

python C:\Users\vinic\Workspace\LPS\SonarAnalysis\Packages\NoveltyDetection\StackedAutoEncoders\sae_train.py --layer 1 --novelty 2 --finetunning 1 --threads 2 --type normal --hiddenNeurons 400 --neuronsVariationStep 50 --modelhash 354a72b3e4e

In [ ]:
analysis.train(layer=1,
               inovelty=0,
               fineTuning=True,
               trainingType="normal",
               hidden_neurons=[1],
               neurons_variation_step=50,
               numThreads=4,
               model_hash=analysis.model_hash)

### Grid Search

In [ ]:
import copy
num_values = 4
gridSearch = [copy.deepcopy(training_params) for i in range(num_values)]
for params in gridSearch:
    params['HyperParameters']['regularization_parameter'] = 2**-i
    print params['HyperParameters']['regularization_parameter']
    i = i - 1
    
gridSearch[3]